In [133]:
# default_exp digdata
# 上面一行用于nbdev中声明本模块的名称。必须是notebook的第一个Cell的第一行。

# digdata

- 描述：抗击新冠病毒（3）-探索在线数据资源
- 功能：本页面用于交互式地探索数据。通过访问网络获取数据，分析和理解网页数据结构，转换为列表格式，用于后续的分析和绘图输出。
- 模块：使用JupyterLab、Python、nbdev等完成。用到的Python模块包括：
  - requests，访问web服务网站。
  - re，正则表达式解析。
  - json，JSON格式解析。
  - BeautifulSoup，HTML格式解析。
  - pprint，格式化输出。
  - pandas，数据表格分析。
- [源码-https://github.com/openthings/anti2020ncov](https://github.com/openthings/anti2020ncov)
- 参考：
  - [JupyterLab-数据实验室](https://my.oschina.net/u/2306127?q=jupyterlab)
  - [文学式编程-nbdev入门教程](https://my.oschina.net/u/2306127/blog/3163740)
  - [抗击新冠病毒（1）-开源软件与数据项目](https://my.oschina.net/u/2306127/blog/3162141)
  - [抗击新冠病毒（2）-基于Jupyter+nbdev的数据分析](https://my.oschina.net/u/2306127/blog/3164083)
  - [抗击新冠病毒（3）-探索在线数据资源](https://my.oschina.net/u/2306127/blog/3164744)

In [159]:
#hide
from nbdev.showdoc import *

In [160]:
#export 
from bs4 import BeautifulSoup
from parser import * #regex_parser
import re
import json
import time
import logging
import datetime
import requests
import pprint 

#### 获取网页数据

In [161]:
#export 
#url = "https://3g.dxy.cn/newh5/view/pneumonia"
url = "https://ncov.dxy.cn/ncovh5/view/pneumonia?from=singlemessage&isappinstalled=0"
headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'
}

In [162]:
#export 
session = requests.session()
session.headers.update(headers)
r = session.get(url)

In [163]:
#export 
#pprint.pprint(r.text)

In [164]:
#export 
#soup = BeautifulSoup(r.content, 'lxml')
#soup

### 提取特定的数据域

In [188]:
# export 
# 分为总体情况、分省情况、省内各市情况、新闻四大类。
overall_information = re.search(r'\{("id".*?)\}', str(soup.find('script', attrs={'id': 'getStatisticsService'})))
province_information = re.search(r'\[(.*?)\]', str(soup.find('script', attrs={'id': 'getListByCountryTypeService1'})))
area_information = re.search(r'\[(.*)\]', str(soup.find('script', attrs={'id': 'getAreaStat'})))
news_information = re.search(r'\[(.*?)\]', str(soup.find('script', attrs={'id': 'getTimelineService'})))

#### 1、总体情况

In [166]:
#pprint.pprint(overall_information.string)

In [167]:
#overall_information.group(0)

In [168]:
#jsall = json.loads(overall_information.group(0))

def overall_parser(overall_information):
    overall_information = json.loads(overall_information.group(0))
    overall_information.pop('id')
    overall_information.pop('createTime')
    overall_information.pop('modifyTime')
    overall_information.pop('imgUrl')
    overall_information.pop('deleted')
    overall_information['countRemark'] = overall_information['countRemark'].replace(' 疑似', '，疑似').replace(' 治愈', '，治愈').replace(' 死亡', '，死亡').replace(' ', '')

In [169]:
#overall_information = json.loads(overall_information.group(0))

#### 2、分省情况

In [170]:
#provinces = json.loads(province_information.group(0))
#provinces

In [171]:
#for province in provinces:
#    print(province['id'],'\t',province['provinceShortName'],'\t',province['tags'])

#### 3、省内各市县情况

In [172]:
#area_information.string

In [217]:
area = json.loads(area_information.group(0))
#area

In [174]:
print("省份\t确诊\t疑似\t治愈\t死亡")
for a in area:
    print(a['provinceName'],'\t',a['confirmedCount'],'\t',a['suspectedCount'],'\t',a['curedCount'],'\t',a['deadCount'])

省份	确诊	疑似	治愈	死亡
湖北省 	 29631 	 0 	 1795 	 871
广东省 	 1151 	 0 	 143 	 1
浙江省 	 1092 	 0 	 201 	 0
河南省 	 1073 	 0 	 169 	 6
湖南省 	 879 	 0 	 186 	 1
安徽省 	 830 	 0 	 73 	 3
江西省 	 771 	 0 	 102 	 1
江苏省 	 492 	 0 	 72 	 0
重庆市 	 468 	 0 	 51 	 2
山东省 	 459 	 0 	 63 	 1
四川省 	 405 	 0 	 76 	 1
黑龙江省 	 331 	 0 	 15 	 7
北京市 	 326 	 0 	 37 	 2
上海市 	 295 	 0 	 44 	 1
福建省 	 261 	 0 	 35 	 0
河北省 	 218 	 0 	 35 	 2
陕西省 	 213 	 0 	 26 	 0
广西壮族自治区 	 210 	 0 	 18 	 1
云南省 	 141 	 0 	 18 	 0
海南省 	 136 	 0 	 19 	 3
山西省 	 119 	 0 	 25 	 0
贵州省 	 109 	 0 	 7 	 1
辽宁省 	 107 	 0 	 12 	 0
天津市 	 91 	 0 	 4 	 1
甘肃省 	 83 	 0 	 16 	 2
吉林省 	 80 	 0 	 12 	 1
内蒙古自治区 	 58 	 0 	 5 	 0
宁夏回族自治区 	 49 	 0 	 13 	 0
新疆维吾尔自治区 	 49 	 0 	 0 	 0
香港 	 36 	 0 	 0 	 1
台湾 	 18 	 0 	 1 	 0
青海省 	 18 	 0 	 3 	 0
澳门 	 10 	 0 	 1 	 0
西藏自治区 	 1 	 0 	 0 	 0


#### 按省提取城市情况

In [220]:
cities = area[0]['cities']

cities

[{'cityName': '武汉',
  'confirmedCount': 16902,
  'suspectedCount': 0,
  'curedCount': 1045,
  'deadCount': 681,
  'locationId': 420100},
 {'cityName': '孝感',
  'confirmedCount': 2541,
  'suspectedCount': 0,
  'curedCount': 81,
  'deadCount': 33,
  'locationId': 420900},
 {'cityName': '黄冈',
  'confirmedCount': 2252,
  'suspectedCount': 0,
  'curedCount': 189,
  'deadCount': 45,
  'locationId': 421100},
 {'cityName': '随州',
  'confirmedCount': 1049,
  'suspectedCount': 0,
  'curedCount': 23,
  'deadCount': 10,
  'locationId': 421300},
 {'cityName': '荆州',
  'confirmedCount': 1045,
  'suspectedCount': 0,
  'curedCount': 45,
  'deadCount': 15,
  'locationId': 421000},
 {'cityName': '襄阳',
  'confirmedCount': 1019,
  'suspectedCount': 0,
  'curedCount': 42,
  'deadCount': 9,
  'locationId': 420600},
 {'cityName': '黄石',
  'confirmedCount': 805,
  'suspectedCount': 0,
  'curedCount': 67,
  'deadCount': 3,
  'locationId': 420200},
 {'cityName': '宜昌',
  'confirmedCount': 749,
  'suspectedCount': 0,

In [226]:
print("城市\t确诊\t疑似\t治愈\t死亡")
for p in area:
    cities = p['cities']
    print("===================================")
    print(p['provinceName'],'\t',p['confirmedCount'],'\t',p['suspectedCount'],'\t',p['curedCount'],'\t',p['deadCount'])
    print("-----------------------------------")
    #pprint.pprint(cities)
    for c in cities:
        #pprint.pprint(c)
        print(c['cityName'],'\t',str(c['confirmedCount']),'\t',str(c['suspectedCount']),'\t',c['curedCount'],'\t',c['deadCount'])

城市	确诊	疑似	治愈	死亡
湖北省 	 29631 	 0 	 1795 	 871
-----------------------------------
武汉 	 16902 	 0 	 1045 	 681
孝感 	 2541 	 0 	 81 	 33
黄冈 	 2252 	 0 	 189 	 45
随州 	 1049 	 0 	 23 	 10
荆州 	 1045 	 0 	 45 	 15
襄阳 	 1019 	 0 	 42 	 9
黄石 	 805 	 0 	 67 	 3
宜昌 	 749 	 0 	 43 	 8
鄂州 	 725 	 0 	 48 	 24
荆门 	 641 	 0 	 63 	 20
咸宁 	 507 	 0 	 44 	 4
十堰 	 481 	 0 	 46 	 0
仙桃 	 416 	 0 	 16 	 7
天门 	 217 	 0 	 10 	 10
恩施州 	 187 	 0 	 24 	 0
潜江 	 85 	 0 	 3 	 2
神农架林区 	 10 	 0 	 6 	 0
广东省 	 1151 	 0 	 143 	 1
-----------------------------------
深圳 	 368 	 0 	 46 	 0
广州 	 313 	 0 	 36 	 0
珠海 	 85 	 0 	 4 	 0
佛山 	 68 	 0 	 4 	 0
东莞 	 60 	 0 	 1 	 0
中山 	 56 	 0 	 11 	 0
惠州 	 50 	 0 	 1 	 0
汕头 	 25 	 0 	 3 	 0
湛江 	 21 	 0 	 6 	 0
江门 	 19 	 0 	 1 	 0
肇庆 	 14 	 0 	 1 	 1
阳江 	 13 	 0 	 1 	 0
梅州 	 13 	 0 	 4 	 0
清远 	 10 	 0 	 3 	 0
茂名 	 10 	 0 	 1 	 0
揭阳 	 7 	 0 	 1 	 0
韶关 	 6 	 0 	 1 	 0
汕尾 	 5 	 0 	 1 	 0
潮州 	 5 	 0 	 0 	 0
河源 	 3 	 0 	 0 	 0
待明确地区 	 0 	 0 	 17 	 0
浙江省 	 1092 	 0 	 201 	 0
------------------

#### 4、新闻列表

In [215]:
news = json.loads(news_information.group(0))
#news

In [213]:
for n in news:
    print(n['id'],'\t',n['infoSource'].strip(),'\t',n['title'].strip())#,n['summary'].strip())

3400 	 人民日报 	 西藏连续11天无新增病例
3379 	 人民日报 	 广东新增确诊31例，累计确诊1151例
3365 	 人民日报 	 江苏新增确诊24例，累计确诊492例
3364 	 人民网 	 江西首次报告死亡病例
3363 	 人民日报 	 湖南新增确诊41例，累计确诊879
3360 	 人民日报 	 浙江新增确诊29例，累计确诊1092例
3359 	 人民网 	 安徽新增51例累计830例
3362 	 贵州卫健委 	 贵州新增确诊病例10例，累计109例
3368 	 河南卫健委 	 河南新增确诊病例40例，累计1073例


## nbdev 适用工具

In [122]:
# 将notebook转化为python的*.py代码，保存到项目名称的子目录中。

from nbdev.export import *
notebook2script()

Converted 00_digdata.ipynb.
Converted 01_getdata.ipynb.
Converted 10_charts.ipynb.
Converted 10_china.ipynb.
Converted index.ipynb.


In [132]:
help(notebook2script)

Help on function notebook2script in module nbdev.export:

notebook2script(fname=None, silent=False, to_dict=False)
    Convert notebooks matching `fname` to modules

